In [5]:
print("Hello World5")


Hello World5


In [1]:
# IMPORT, INIT DB & TABLES
import os
from sqlalchemy import create_engine, Table, Column, \
    Integer, String, Float, ForeignKey, MetaData, \
    insert, select, update, delete
from sqlalchemy.orm import declarative_base, sessionmaker

# # Try also:
from pydantic import EmailStr

# Setup Database

DATABASE_FILE = 'ecommerce.db' # file name
DATABASE_URL = f"sqlite:///{DATABASE_FILE}" # def db url
engine = create_engine(DATABASE_URL)
metadata = MetaData()

# Table Definitions

users = Table(
    'users', metadata,
    Column('id', Integer, primary_key=True),
    Column('name', String),
    Column('email', String, unique=True)
)

products = Table(
    'products', metadata,
    Column('id', Integer, primary_key=True),
    Column('name', String)
)

orders = Table(
    'orders', metadata,
    Column('id', Integer, primary_key=True),
    Column('user_id', Integer, ForeignKey('users.id'))
)

order_items = Table(
    'order_items', metadata,
    Column('id', Integer, primary_key=True),
    Column('order_id', Integer, ForeignKey('orders.id'), nullable=False),
    Column('product_id', Integer, ForeignKey('products.id')),
    Column('quantity', Integer, nullable=False)
)

metadata.create_all(engine)
print(f"database '{DATABASE_FILE}' and its tables created successfully.")

database 'ecommerce.db' and its tables created successfully.


# INSERTION

In [2]:
def insert_user(name, email):
    """res = result, stmt = statement, conn = connection"""
    with engine.connect() as conn:
        stmt = insert(users).values(name=name, email=email)
        res = conn.execute(stmt)
        conn.commit()
        print(f"✅ inserted user with ID: {res.lastrowid}")

insert_user('Wilson','wilson@hutapea.com')
insert_user('Dwi','dwi@naibaho.com')

def insert_product(name:str):
    with engine.connect() as conn:
        stmt = insert(products).values(name=name)
        res = conn.execute(stmt)
        conn.commit()
        print(f"✅ inserted product with ID: {res.lastrowid}")

def insert_products(names:list):
    for n in names:
        insert_product(n['name'])

insert_products([
    {'name':'Asus Laptop'},
    {'name':'Lenovo Laptop'},
    {'name':'Iphone Promag'}
])

def insert_order_item(order_id:int, product_id:int, quantity:int):
    with engine.connect() as conn:
        stmt = insert(order_items).values(order_id=order_id, product_id=product_id, quantity=quantity)
        conn.execute(stmt)
        conn.commit()

def insert_order_items(order_id:int, product_ids:list, quantities:list):
    for p,q in zip(product_ids, quantities):
        insert_order_item(order_id,p,q)


def insert_new_order(user_id:int, product_ids:list, quantities:list):
    with engine.connect() as conn:
        stmt = insert(orders).values(user_id=user_id)
        res = conn.execute(stmt)
        conn.commit()

        current_order_id = res.lastrowid
        stmt2 = insert_order_items(current_order_id, product_ids, quantities)
        res = conn.execute(stmt2)

        conn.commit()
        print(f"✅ inserted order with ID: {current_order_id}. Details:\nUser ID: {user_id}")
        i = 1
        for p,q in product_ids,quantities:
            print(f"    {i}. Ordered {q}x {p}")
            i+=1

# class Order_item:
#     product_id:str
#     quantity:int

#     def __init__(self, product_id:int, quantity:int):
#         self.product_id=product_id
#         self.quantity=quantity
# order_item = Order_item()
# insert_new_order(1,order_item)

insert_new_order(1,[1],[1])
insert_new_order(2,[2,3],[1,1])



✅ inserted user with ID: 1
✅ inserted user with ID: 2
✅ inserted product with ID: 1
✅ inserted product with ID: 2
✅ inserted product with ID: 3


ValueError: not enough values to unpack (expected 2, got 1)

# READ

### Define func to Read all users

In [ ]:
def get_all_users():
    with engine.connect() as conn:
        stmt = select(users)
        res = conn.execute(stmt)

        all_users = res.fetchall()
    return all_users

def print_all_users():
    all_users = get_all_users()
    print("\nAll users:")
    for row in all_users:
        print(f"ID: {row.id}, Name: {row.name}, Email: {row.email}")

print_all_users()


All users:
ID: 1, Name: Wilson, Email: wilson@hutapea.com
ID: 2, Name: Dwi, Email: dwi@naibaho.com


# $UPDATE$

In [ ]:
with engine.connect() as conn:
    stmt = update(users).where(users.c.email.ilike("%WILSON%")).values(email='wilson5@hutapea.com')
    conn.execute(stmt)
    conn.commit()

    # verify the update
    stmt2 = select(users)
    res2 = conn.execute(stmt2)
    for r in res2:
        print(r)

with engine.connect() as conn:
    stmt = delete(products).where(products.c.name == "Iphone Promag")
    conn.execute(stmt)
    conn.commit()

    # verify the delete
    stmt2 = select(products)
    res = conn.execute(stmt2)
    print("\nUpdated Products")
    for r in res:
        print(r)

(1, 'Wilson', 'wilson5@hutapea.com')
(2, 'Dwi', 'dwi@naibaho.com')

Updated Products
(1, 'Asus Laptop')
(2, 'Lenovo Laptop')
